In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [2]:
pandf = fdr.DataReader("KS11","2009.01.01","2019-01-01")

In [3]:
# convert nparray
nparr = pandf.values
nparr.astype('float32')
print(nparr)

[[ 1.15740e+03  1.13287e+03  1.15740e+03  1.11884e+03  3.40780e+08
   2.93000e-02]
 [ 1.17357e+03  1.18098e+03  1.18522e+03  1.16666e+03  4.14760e+08
   1.40000e-02]
 [ 1.19428e+03  1.18352e+03  1.20359e+03  1.17994e+03  4.44200e+08
   1.76000e-02]
 ...
 [ 2.02801e+03  2.02881e+03  2.03783e+03  2.01428e+03  3.21500e+08
  -1.31000e-02]
 [ 2.02844e+03  2.03209e+03  2.03557e+03  2.02139e+03  3.98020e+08
   2.00000e-04]
 [ 2.04104e+03  2.03670e+03  2.04697e+03  2.03541e+03  3.52680e+08
   6.20000e-03]]


In [4]:
# normalization
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)

In [5]:
nptf=nptf[:-1]

In [6]:
# split train, test
y=np.where(pandf["Change"].shift(-1).dropna()>0,1,0)
train_size = int(len(nptf) * 0.7)
y_train=y[:train_size]
y_test=y[train_size:]
X_train = nptf[:train_size]
X_test = nptf[train_size:]

In [7]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [10]:
# simple lstm network learning
model = Sequential()
model.add(SimpleRNN(36, input_shape=(1, 6)))
for i in range(5):
    model.add(Dense(36,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, batch_size=16,validation_split=0.1)
model.evaluate(X_test,y_test)

Train on 1557 samples, validate on 174 samples
Epoch 1/100
1557/1557 [==============================] - 1s 425us/step - loss: 0.6946 - acc: 0.4978 - val_loss: 0.7071 - val_acc: 0.4828
Epoch 2/100
1557/1557 [==============================] - 0s 261us/step - loss: 0.6975 - acc: 0.5042 - val_loss: 0.6932 - val_acc: 0.4828
Epoch 3/100
1557/1557 [==============================] - 0s 263us/step - loss: 0.6931 - acc: 0.4984 - val_loss: 0.6983 - val_acc: 0.4828
Epoch 4/100
1557/1557 [==============================] - 0s 264us/step - loss: 0.6922 - acc: 0.5228 - val_loss: 0.6926 - val_acc: 0.5172
Epoch 5/100
1557/1557 [==============================] - 0s 261us/step - loss: 0.6942 - acc: 0.4971 - val_loss: 0.6979 - val_acc: 0.4828
Epoch 6/100
1557/1557 [==============================] - 0s 262us/step - loss: 0.6943 - acc: 0.5080 - val_loss: 0.6956 - val_acc: 0.4828
Epoch 7/100
1557/1557 [==============================] - 0s 262us/step - loss: 0.6945 - acc: 0.5093 - val_loss: 0.6943 - val_acc: 0

1557/1557 [==============================] - 0s 268us/step - loss: 0.6909 - acc: 0.5267 - val_loss: 0.6961 - val_acc: 0.4828
Epoch 61/100
1557/1557 [==============================] - 0s 259us/step - loss: 0.6916 - acc: 0.5215 - val_loss: 0.6908 - val_acc: 0.5345
Epoch 62/100
1557/1557 [==============================] - 0s 266us/step - loss: 0.6910 - acc: 0.5234 - val_loss: 0.6920 - val_acc: 0.5172
Epoch 63/100
1557/1557 [==============================] - 0s 261us/step - loss: 0.6917 - acc: 0.5093 - val_loss: 0.6930 - val_acc: 0.4828
Epoch 64/100
1557/1557 [==============================] - 0s 265us/step - loss: 0.6910 - acc: 0.5292 - val_loss: 0.6919 - val_acc: 0.5172
Epoch 65/100
1557/1557 [==============================] - 0s 261us/step - loss: 0.6905 - acc: 0.5337 - val_loss: 0.6912 - val_acc: 0.5460
Epoch 66/100
1557/1557 [==============================] - 0s 266us/step - loss: 0.6905 - acc: 0.5337 - val_loss: 0.6915 - val_acc: 0.5000
Epoch 67/100
1557/1557 [=======================

[0.6956148480148007, 0.5]

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,np.where(model.predict(X_train)>0.5,1,0))

0.534950895436164